In [3]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

In [35]:
fake = Faker('pt_BR')  # Dados em português

In [36]:
# Conexão com o Postgres
conn = psycopg2.connect(
    dbname="dw_vendas",
    user="postgres",
    password="rubick",  
    host="localhost"
)
cursor = conn.cursor()

In [37]:
# --- 1. Ingestão de CLIENTES ---
for i in range(1, 101):  # 100 clientes
    try:
        cursor.execute(
            """
            INSERT INTO dw_vendas.clientes_bronze (cliente_id, nome, cidade, segmento)
            VALUES (%s, %s, %s, %s)
            """,
            (i, fake.name(), fake.city(), random.choice(["Varejo", "Atacado", "Online"]))
        )
    except Exception as e:
        print(f"Erro ao inserir cliente {i}: {e}")
        conn.rollback()

In [38]:
# --- 2. Ingestão de PRODUTOS ---
categorias = ["Eletrônicos", "Roupas", "Alimentos", "Livros", "Cosméticos"]
for i in range(1, 51):  # 50 produtos
    cursor.execute(
        """
        INSERT INTO dw_vendas.produtos_bronze (produto_id, nome, categoria, preco)
        VALUES (%s, %s, %s, %s)
        """,
        (i, fake.word().capitalize() + " " + fake.word(), random.choice(categorias), round(random.uniform(10, 500), 2))
    )

In [39]:
# --- 3. Ingestão de LOJAS ---
nomes_lojas = ["Super", "Mega", "Hiper", "Ultra", "Maxi"]  # Prefixos para nomes de lojas
for i in range(1, 21):  # 20 lojas
    nome_loja = f"{random.choice(nomes_lojas)} {fake.word().capitalize()}"
    cursor.execute(
        """
        INSERT INTO dw_vendas.lojas_bronze (loja_id, nome, cidade, tamanho)
        VALUES (%s, %s, %s, %s)
        """,
        (i, 
         nome_loja,  # Ex: "Super Eletrônicos"
         fake.city(),
         random.choice(["Pequena", "Média", "Grande"])
    ))

In [40]:
# --- 4. Ingestão de VENDAS (agora com loja_id) ---
for i in range(1, 1001):  # 1000 vendas
    data_venda = fake.date_between(start_date='-1y', end_date='today')
    cliente_id = random.randint(1, 100)
    produto_id = random.randint(1, 50)
    loja_id = random.randint(1, 20)
    quantidade = random.randint(1, 5)
    preco = round(random.uniform(10, 500), 2)
    total = quantidade * preco

    try:
        cursor.execute(
            """
            INSERT INTO dw_vendas.vendas_bronze (venda_id, cliente_id, produto_id, quantidade, data_venda, total)
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (i, cliente_id, produto_id, quantidade, data_venda, total)
        )
    except Exception as e:
        print(f"Erro ao inserir venda {i}: {e}")
        conn.rollback()

In [ ]:
# Commit e fechamento
conn.commit()
cursor.close()
conn.close()
print("Dados ingeridos com sucesso nas tabelas Bronze!")

Dados ingeridos com sucesso nas tabelas Bronze (com lojas)!
